## Deep Learning Project 1 

Comparing digits

### TODO Next : 

- Maybe we are already using Weight Sharing ? **WE DO**
- Maybe get better performances ? 
- Add dropout layer and stuff like this
- Add Auxilary losses (also return CNNs results and use them with a loss function, similar to comparisson net) **Done**
- Benchmark **Function Done** 
- Write report **Started** 

In [1]:
import torch
import math
import dlc_practical_prologue as prologue
from torch import optim
from torch import Tensor
from torch import nn
from torch.nn import functional as F

In [2]:
# Generate the train and test sets.
N = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)

In [18]:
# This model performs each digit classification with 2 different CNNs (so no weight sharing)        
class Base_Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layers that handle digit classification with first CNN
        self.conv1_1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2_1 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1_1 = nn.Linear(256, 200)
        self.fc2_1 = nn.Linear(200, 10)
        
        # Layers that handle digit classification with second CNN
        self.conv1_2 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2_2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1_2 = nn.Linear(256, 200)
        self.fc2_2 = nn.Linear(200, 10)
        
        # Layers that handle comparisson 
        self.fc3 = nn.Linear(20, 300)
        self.fc4 = nn.Linear(300, 300)
        self.fc5 = nn.Linear(300, 2)
        
    def cnn1(self, x):
        x = F.relu(F.max_pool2d(self.conv1_1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2_1(x), kernel_size=2))
        x = F.relu(self.fc1_1(x.view(-1, 256)))
        x = self.fc2_1(x)
        return x
    
    def cnn2(self, x):
        x = F.relu(F.max_pool2d(self.conv1_2(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2_2(x), kernel_size=2))
        x = F.relu(self.fc1_2(x.view(-1, 256)))
        x = self.fc2_2(x)
        return x
    
    def mlp(self, x):
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
    def forward(self, x):
        s = x.shape
        input_1 = x[:,0,:,:].reshape([s[0],1,s[2],s[3]])
        input_2 = x[:,1,:,:].reshape([s[0],1,s[2],s[3]])
        
        output_1 = self.cnn1(input_1)
        output_2 = self.cnn2(input_2)
        
        concatenated = torch.cat((output_1, output_2), 1)
        
        comparison = self.mlp(concatenated)
        return comparison   

In [4]:
# Model Definition 
  
class Weight_Sharing_Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layers that handle digit classification 
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        
        # Layers that handle comparisson 
        self.fc3 = nn.Linear(20, 300)
        self.fc4 = nn.Linear(300, 300)
        self.fc5 = nn.Linear(300, 2)
        
    def cnn(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x
    
    def mlp(self, x):
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
    def forward(self, x):
        s = x.shape
        input_1 = x[:,0,:,:].reshape([s[0],1,s[2],s[3]])
        input_2 = x[:,1,:,:].reshape([s[0],1,s[2],s[3]])
        
        output_1 = self.cnn(input_1)
        output_2 = self.cnn(input_2)
        
        concatenated = torch.cat((output_1, output_2), 1)
        
        comparison = self.mlp(concatenated)
        return comparison   

In [5]:
def train_model_base_ws(model, train_input, train_target, mini_batch_size, nb_epochs = 100, use_optimizer= None, _print=False):
    criterion = nn.CrossEntropyLoss()
    eta = 1e-3
    if use_optimizer == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=eta)
    if use_optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=eta)
    for e in range(nb_epochs):
        acc_loss = 0

        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            target = train_target.narrow(0, b, mini_batch_size).long()
            loss = criterion(output, target)
            acc_loss = acc_loss + loss.item()
 
            model.zero_grad()
            loss.backward()
            
            if use_optimizer != None :
                optimizer.step()
            else :
                with torch.no_grad():
                    for p in model.parameters():
                        p -= eta * p.grad
        if _print:
            print(e, acc_loss)
        
def compute_nb_errors_base_ws(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        for k in range(mini_batch_size):
            if target[b + k, predicted_classes[k]] <= 0:
                nb_errors = nb_errors + 1

    return nb_errors

In [ ]:
model_total = No_Weight_Sharing_Net()

train_target_one_hot = prologue.convert_to_one_hot_labels(train_input, train_target)
train_model_ws(model_total, train_input, train_target, mini_batch_size=250, nb_epochs=25, use_optimizer="adam")

test_target_total = prologue.convert_to_one_hot_labels(test_input, test_target)
nb_test_errors = compute_nb_errors_ws(model_total, test_input, test_target_total, mini_batch_size=250)
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))

In [6]:
model_total = Weight_Sharing_Net()

train_model_ws(model_total, train_input, train_target, mini_batch_size=250, nb_epochs=25, use_optimizer="adam")

test_target_total = prologue.convert_to_one_hot_labels(test_input, test_target)
nb_test_errors = compute_nb_errors_ws(model_total, test_input, test_target_total, mini_batch_size=250)
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))

test error Net 15.50% 155/1000


In [7]:
class Auxiliary_Loss_Weight_Sharing_Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layers that handle digit classification 
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        
        # Layers that handle comparisson 
        self.fc3 = nn.Linear(20, 300)
        self.fc4 = nn.Linear(300, 300)
        self.fc5 = nn.Linear(300, 2)
        
    def cnn(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x
    
    def mlp(self, x):
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
    def forward(self, x):
        s = x.shape
        
        input_1 = x[:,0,:,:].reshape([s[0],1,s[2],s[3]])
        input_2 = x[:,1,:,:].reshape([s[0],1,s[2],s[3]])
        
        output_1 = self.cnn(input_1)
        output_2 = self.cnn(input_2)
        
        concatenated = torch.cat((output_1, output_2), 1)
        
        comparison = self.mlp(concatenated)
        return output_1, output_2, comparison  

        
def train_model_auxiliary_loss(model, train_input, train_target, train_classes, mini_batch_size, nb_epochs = 100, use_optimizer= None, _print=False):
    criterion_auxilary = nn.CrossEntropyLoss()
    criterion_final = nn.CrossEntropyLoss()
    
    eta = 1e-3
    if use_optimizer == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=eta)
    if use_optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=eta)
    for e in range(nb_epochs):
        acc_loss = 0

        for b in range(0, train_input.size(0), mini_batch_size):
            digit_1, digit_2, comparison = model(train_input.narrow(0, b, mini_batch_size))
            
            target_comparison = train_target.narrow(0, b, mini_batch_size).long()
            
            target_digit_1, target_digit_2 = train_classes.narrow(0, b, mini_batch_size)[:,0], train_classes.narrow(0, b, mini_batch_size)[:,1]
            loss1 = criterion_auxilary(digit_1, target_digit_1)
            loss2 = criterion_auxilary(digit_2, target_digit_2)
            loss3 = criterion_final(comparison, target_comparison)
            acc_loss = acc_loss + loss1.item() + loss2.item() + loss3.item()
 
            model.zero_grad()
            loss1.backward(retain_graph=True)
            loss2.backward(retain_graph=True)
            loss3.backward()
            
            if use_optimizer != None :
                optimizer.step()
            else :
                with torch.no_grad():
                    for p in model.parameters():
                        p -= eta * p.grad
        if _print :
            print(e, acc_loss)
            
def compute_nb_errors_auxilary_loss(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        _, _, output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        for k in range(mini_batch_size):
            if target[b + k, predicted_classes[k]] <= 0:
                nb_errors = nb_errors + 1

    return nb_errors

In [8]:
model_auxiliary = Auxiliary_Loss_Weight_Sharing_Net()


train_model_auxiliary_loss(model_auxiliary, train_input, train_target, train_classes, mini_batch_size=250, nb_epochs=25, use_optimizer="adam")


test_target_total = prologue.convert_to_one_hot_labels(test_input, test_target)
nb_test_errors = compute_nb_errors_auxilary_loss(model_auxiliary, test_input, test_target_total, mini_batch_size=250)
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))

test error Net 10.10% 101/1000


In [9]:
def benchmark_model(model, train_function, evaluate_function, nb_trials=20, N=1000, mini_batch_size=250, nb_epochs=25, model_requires_target_and_classes=False, _print=False):
    # Benchmark of the basic network with Adam optimizer
    performances = []
    for trial in range(nb_trials):

        # Generate Data 
        train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)
        test_target_one_hot = prologue.convert_to_one_hot_labels(test_input, test_target)

        # Define the model 
        model_total = model()

        # Train the model
        if model_requires_target_and_classes : 
            train_function(model_total, train_input, train_target, train_classes, mini_batch_size=mini_batch_size,
                           nb_epochs=nb_epochs, use_optimizer="adam", _print=_print)
        else :
            train_function(model_total, train_input, train_target, mini_batch_size=mini_batch_size,
                           nb_epochs=nb_epochs, use_optimizer="adam", _print=_print)

        # Evaluate performances 
        nb_test_errors = evaluate_function(model_total, test_input, test_target_one_hot, mini_batch_size=mini_batch_size)
        print('test error Net trial {:d} {:0.2f}% {:d}/{:d}'.format(trial, (100 * nb_test_errors) / test_input.size(0),
                                                              nb_test_errors, test_input.size(0)))
        performances.append(nb_test_errors)

    mean_perf = 100 * sum(performances) / (N * nb_trials)
    print(f"Average precision of this architecture {mean_perf}%")
    
    std_dev = math.sqrt(sum(list(map(lambda x : x - mean_perf,performances))))/nb_trials
    print(f"With standard deviation of  {std_dev}")

In [19]:
print("Benchmark of the model with no Weight Sharing")
benchmark_model(Base_Net, train_model_base_ws, compute_nb_errors_base_ws)

Benchmark of the model with no Weight Sharing
test error Net trial 0 18.90% 189/1000
test error Net trial 1 15.80% 158/1000
test error Net trial 2 18.70% 187/1000
test error Net trial 3 19.80% 198/1000
test error Net trial 4 14.10% 141/1000
test error Net trial 5 16.40% 164/1000
test error Net trial 6 16.80% 168/1000
test error Net trial 7 18.30% 183/1000
test error Net trial 8 18.10% 181/1000
test error Net trial 9 16.30% 163/1000
test error Net trial 10 16.60% 166/1000
test error Net trial 11 16.30% 163/1000
test error Net trial 12 17.20% 172/1000
test error Net trial 13 17.20% 172/1000
test error Net trial 14 18.60% 186/1000
test error Net trial 15 14.30% 143/1000
test error Net trial 16 15.30% 153/1000
test error Net trial 17 18.60% 186/1000
test error Net trial 18 20.20% 202/1000
test error Net trial 19 17.00% 170/1000
Average precision of this architecture 17.225%
With standard deviation of  2.784106678990588


In [11]:
print("Benchmark of the model with Weight Sharing")
benchmark_model(Weight_Sharing_Net, train_model_base_ws, compute_nb_errors_base_ws)

Benchmark of the model with Weight Sharing
test error Net trial 0 15.90% 159/1000
test error Net trial 1 15.00% 150/1000
test error Net trial 2 15.60% 156/1000
test error Net trial 3 16.70% 167/1000
test error Net trial 4 15.70% 157/1000
test error Net trial 5 19.20% 192/1000
test error Net trial 6 17.60% 176/1000
test error Net trial 7 14.70% 147/1000
test error Net trial 8 14.70% 147/1000
test error Net trial 9 15.70% 157/1000
test error Net trial 10 17.10% 171/1000
test error Net trial 11 17.00% 170/1000
test error Net trial 12 14.80% 148/1000
test error Net trial 13 16.00% 160/1000
test error Net trial 14 16.40% 164/1000
test error Net trial 15 14.90% 149/1000
test error Net trial 16 17.60% 176/1000
test error Net trial 17 16.40% 164/1000
test error Net trial 18 15.50% 155/1000
test error Net trial 19 16.50% 165/1000
Average precision of this architecture 16.15%
With standard deviation of  2.6958301133417137


In [13]:
print("Benchmark of the model with Weight Sharing and an auxiliary loss ")
benchmark_model(Auxiliary_Loss_Weight_Sharing_Net, train_model_auxiliary_loss, compute_nb_errors_auxilary_loss, model_requires_target_and_classes=True)


Benchmark of the model with Weight Sharing and an auxiliary loss 
test error Net trial 0 10.90% 109/1000
test error Net trial 1 12.70% 127/1000
test error Net trial 2 9.40% 94/1000
test error Net trial 3 10.60% 106/1000
test error Net trial 4 12.30% 123/1000
test error Net trial 5 10.40% 104/1000
test error Net trial 6 11.50% 115/1000
test error Net trial 7 9.60% 96/1000
test error Net trial 8 10.30% 103/1000
test error Net trial 9 10.50% 105/1000
test error Net trial 10 9.50% 95/1000
test error Net trial 11 13.30% 133/1000
test error Net trial 12 10.60% 106/1000
test error Net trial 13 11.20% 112/1000
test error Net trial 14 10.50% 105/1000
test error Net trial 15 13.20% 132/1000
test error Net trial 16 11.70% 117/1000
test error Net trial 17 11.50% 115/1000
test error Net trial 18 9.20% 92/1000
test error Net trial 19 9.00% 90/1000
Average precision of this architecture 10.895%
With standard deviation of  2.2142154366727733


In [14]:
class Auxiliary_Loss_Net_Dropout(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layers that handle digit classification 
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.dropout_1 = nn.Dropout(p=0.1)
        self.dropout_2 = nn.Dropout(p=0.1)
        self.dropout_3 = nn.Dropout(p=0.1)
        # Layers that handle comparisson 
        self.fc3 = nn.Linear(20, 300)
        self.fc4 = nn.Linear(300, 300)
        self.fc5 = nn.Linear(300, 2)
        
    def cnn(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = self.dropout_1(x)
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = self.dropout_2(x)
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.dropout_3(x)
        x = self.fc2(x)
        return x
    
    def mlp(self, x):
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
    def forward(self, x):
        s = x.shape
        
        input_1 = x[:,0,:,:].reshape([s[0],1,s[2],s[3]])
        input_2 = x[:,1,:,:].reshape([s[0],1,s[2],s[3]])
        
        output_1 = self.cnn(input_1)
        output_2 = self.cnn(input_2)
        
        concatenated = torch.cat((output_1, output_2), 1)
        
        comparison = self.mlp(concatenated)
        return output_1, output_2, comparison  
    

In [15]:
benchmark_model(Auxiliary_Loss_Net_Dropout, train_model_auxiliary_loss, compute_nb_errors_auxilary_loss, model_requires_target_and_classes=True, nb_epochs=50)


test error Net trial 0 9.70% 97/1000
test error Net trial 1 7.00% 70/1000
test error Net trial 2 9.00% 90/1000
test error Net trial 3 8.50% 85/1000
test error Net trial 4 7.10% 71/1000
test error Net trial 5 9.60% 96/1000
test error Net trial 6 7.90% 79/1000
test error Net trial 7 7.50% 75/1000
test error Net trial 8 8.20% 82/1000
test error Net trial 9 9.40% 94/1000
test error Net trial 10 7.70% 77/1000
test error Net trial 11 9.50% 95/1000
test error Net trial 12 10.70% 107/1000
test error Net trial 13 6.80% 68/1000
test error Net trial 14 10.50% 105/1000
test error Net trial 15 10.20% 102/1000
test error Net trial 16 9.20% 92/1000
test error Net trial 17 7.80% 78/1000
test error Net trial 18 9.10% 91/1000
test error Net trial 19 9.70% 97/1000
Average precision of this architecture 8.755%
With standard deviation of  1.984880349038702
